### 作者を分類

In [6]:
import urllib.request as req
import zipfile
import os.path
import MeCab
from gensim import models

# MeCabの初期化
mecab = MeCab.Tagger()
mecab.parse('')

# 保存したDoc2Vec学習モデルを読み込む
model = models.Doc2Vec.load('aozora.model')

# 分類用のZipファイルを開き、中の文書を取得する
def read_book(url, zipname):
    if not os.path.exists(zipname):
        req.urlretrieve(url, zipname)
        
    with zipfile.ZipFile(zipname,'r') as zf:
        for filename in zf.namelist():
            with zf.open(filename,'r') as f:
                return f.read().decode('shift-jis')
            
# 引数のテキストを分かち書きして配列にする
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(',')[0]
        if hinshi in ['名詞']:
            wakati_words.append(node.surface)
        elif hinshi in ['動詞', '形容詞']:
            wakati_words.append(node.feature.split(',')[6])
        node = node.next
    return wakati_words

# 引数のタイトル、URLの作品を分類する
def similar(title, url):
    zipname = url.split('/')[-1]
    
    words = read_book(url, zipname)
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("--- 「" + title + '」と似た作品は？ ---')
    print(model.docvecs.most_similar([vector],topn=3))
    print('')
    
# 各作家の作品を一つずつ分類
similar('宮沢 賢治:よだかの星',
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar('夏目 漱石:一夜',
        "https://www.aozora.gr.jp/cards/000148/files/1086_ruby_5742.zip")

similar('ポー エドガー・アラン:マリー・ロジェエの怪事件',
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar('紫式部:源氏物語 06 末摘花',
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

--- 「宮沢 賢治:よだかの星」と似た作品は？ ---
[('夏目 漱石:草枕', 0.9991685152053833), ('夏目 漱石:虞美人草', 0.9991037845611572), ('ポー エドガー・アラン:群集の人', 0.9988319873809814)]

--- 「夏目 漱石:一夜」と似た作品は？ ---
[('ポー エドガー・アラン:群集の人', 0.9959757328033447), ('夏目 漱石:虞美人草', 0.99506014585495), ('夏目 漱石:草枕', 0.9930856227874756)]

--- 「ポー エドガー・アラン:マリー・ロジェエの怪事件」と似た作品は？ ---
[('ポー エドガー・アラン:ウィリアム・ウィルスン', 0.9944115877151489), ('ポー エドガー・アラン:落穴と振子', 0.9936679601669312), ('ポー エドガー・アラン:沈黙', 0.9926974773406982)]

--- 「紫式部:源氏物語 06 末摘花」と似た作品は？ ---
[('紫式部:源氏物語 04 夕顔', 0.9999306797981262), ('宮澤 賢治:セロ弾きのゴーシュ', 0.9999257326126099), ('紫式部:源氏物語 01 桐壺', 0.9998944997787476)]

